In [97]:
import pandas as pd
import requests
import splinter
from bs4 import BeautifulSoup as bs

In [2]:
news_url = 'https://mars.nasa.gov/news/'

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = splinter.Browser('chrome', **executable_path, headless=False)

In [4]:
browser.visit(news_url)

In [5]:
soup = bs(browser.html, 'html.parser')

In [6]:
news_title = soup.body.find(class_='content_title').text
news_p = soup.body.find(class_='article_teaser_body').text

In [7]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [8]:
browser.visit(img_url)

In [9]:
browser.click_link_by_id('full_image')

In [10]:
soup = bs(browser.html, 'html.parser')

In [16]:
img_el = soup.body.find('img', class_='fancybox-image')

In [22]:
featured_img_url = f"{img_url}{img_el.attrs['src']}"

In [23]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'

In [24]:
browser.visit(twitter_url)

In [25]:
soup = bs(browser.html, 'html.parser')

In [35]:
mars_weather = soup.body.find(class_='js-tweet-text').text

In [36]:
facts_url = 'http://space-facts.com/mars/'

In [37]:
browser.visit(facts_url)

In [44]:
facts_df = pd.read_html(facts_url)[0].rename(columns={0:"key", 1:"value"})

In [49]:
facts_html = facts_df.to_html()

In [50]:
hems_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [125]:
browser.visit(hems_url)

In [53]:
soup = bs(browser.html, 'html.parser')

In [81]:
hem_titles = []
for i in soup.body.findAll(class_='product-item'):
    j = i.text.split(' ')[:-1]
    
    if(len(j) > 0):
        k = ' '.join(list(filter(lambda a: a != '', j)))
        hem_titles.append(k)

In [126]:
hem_urls = []
for i in hem_titles:
    browser.click_link_by_partial_text(i)
    soup = bs(browser.html, 'html.parser')
    hem_urls.append(soup.find(class_='downloads').a.attrs['href'])
    browser.visit(hems_url)    

In [135]:
hemisphere_image_urls = []
for i in range(4):
    hemisphere_image_urls.append({'title':hem_titles[i], 'img_url':hem_urls[i]})

In [137]:
browser.quit()